<a href="https://colab.research.google.com/github/MovinduSenura/Chest-Xray-Pneumonia-Detection/blob/dev-oshan/notebooks/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

if not os.path.exists("Chest-Xray-Pneumonia-Detection"):
    !git clone https://github.com/MovinduSenura/Chest-Xray-Pneumonia-Detection.git

Cloning into 'Chest-Xray-Pneumonia-Detection'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 91 (delta 12), reused 5 (delta 5), pack-reused 69 (from 2)
Receiving objects: 100% (91/91), 45.37 KiB | 3.24 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [ ]:
%cd Chest-Xray-Pneumonia-Detection

/content/Chest-Xray-Pneumonia-Detection


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_DIR = '/content/drive/MyDrive/chest_xray'


In [ ]:
from notebooks.preprocessing_2 import create_generators
train_gen, val_gen, test_gen = create_generators(DATA_DIR)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
!pip install tensorflow keras matplotlib numpy

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

# Load ResNet50 base (without top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)  # Dropout for regularization
predictions = Dense(1, activation='sigmoid')(x)  # binary classification

model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze last few layers for fine-tuning
for layer in base_model.layers[:-30]:
    layer.trainable = False
for layer in base_model.layers[-30:]:
    layer.trainable = True


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# === Compile with smaller LR and regularization ===
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# === Add callbacks ===
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, mode='max', restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1),
    ModelCheckpoint('/content/drive/MyDrive/Chest_Xray_Model/best_model.keras',
                    monitor='val_accuracy', save_best_only=True, verbose=1)
]


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 25,686,913 (97.99 MB)

 Trainable params: 2,099,201 (8.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 2755s 17s/step - accuracy: 0.6838 - loss: 0.6352 - val_accuracy: 0.5000 - val_loss: 0.8600
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 121s 743ms/step - accuracy: 0.7457 - loss: 0.5269 - val_accuracy: 0.5000 - val_loss: 0.7606
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 119s 732ms/step - accuracy: 0.7481 - loss: 0.4971 - val_accuracy: 0.5625 - val_loss: 0.7758
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 121s 745ms/step - accuracy: 0.7668 - loss: 0.4609 - val_accuracy: 0.5625 - val_loss: 0.6821
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 119s 729ms/step - accuracy: 0.7747 - loss: 0.4433 - val_accuracy: 0.5625 - val_loss: 0.7363


In [ ]:
from tensorflow.keras.models import load_model

# Load best model weights before final evaluation
best_model = load_model('/content/drive/MyDrive/Chest_Xray_Model/best_model.keras')

val_loss, val_acc = best_model.evaluate(val_gen)
test_loss, test_acc = best_model.evaluate(test_gen)

print(f"\n✅ Best Validation Accuracy: {val_acc:.4f}")
print(f"✅ Best Test Accuracy: {test_acc:.4f}")

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.5000 - loss: 0.7421
✅ Validation Accuracy: 0.50
✅ Validation Loss: 0.74
20/20 ━━━━━━━━━━━━━━━━━━━━ 318s 16s/step - accuracy: 0.2984 - loss: 0.8676

✅ Test Accuracy: 0.625
✅ Test Loss: 0.668


In [ ]:
#Plot Training Accuracy & Loss Curves

import matplotlib.pyplot as plt

# Plot training & validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training & Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training & validation loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Get predictions
y_pred = model.predict(test_gen)
y_pred_classes = (y_pred > 0.5).astype("int32")

# True labels
y_true = test_gen.classes

# Confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Normal', 'Pneumonia'])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Precision, Recall, F1-score
from sklearn.metrics import classification_report

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred_classes, target_names=['Normal', 'Pneumonia']))

In [ ]:
model.save('/content/drive/MyDrive/Chest_Xray_Model/ResNet50_model.keras')


In [ ]:
import os
import csv
from datetime import datetime
import matplotlib.pyplot as plt

# === SETUP SAVE PATHS ===
base_dir = '/content/drive/MyDrive/Chest_Xray_Model/Results'
os.makedirs(base_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
run_folder = os.path.join(base_dir, f"run_{timestamp}")
os.makedirs(run_folder, exist_ok=True)

# === SAVE PLOTS TO RUN FOLDER ===

# Training & Validation Accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training & Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
acc_plot_path = os.path.join(run_folder, "accuracy_plot.png")
plt.savefig(acc_plot_path)
plt.close()

# Training & Validation Loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
loss_plot_path = os.path.join(run_folder, "loss_plot.png")
plt.savefig(loss_plot_path)
plt.close()

# Confusion Matrix
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
cm_plot_path = os.path.join(run_folder, "confusion_matrix.png")
plt.savefig(cm_plot_path)
plt.close()

# === SAVE MODEL (versioned) ===
model_path = os.path.join(run_folder, "ResNet50_model.keras")
model.save(model_path)

# === RECORD RESULTS TO CSV ===
csv_file = os.path.join(base_dir, "training_log.csv")

fieldnames = [
    "Timestamp",
    "Epochs",
    "Train_Accuracy",
    "Val_Accuracy",
    "Val_Loss",
    "Test_Accuracy",
    "Test_Loss",
    "Run_Folder_Path"
]

file_exists = os.path.isfile(csv_file)

with open(csv_file, mode='a', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    if not file_exists:
        writer.writeheader()
    writer.writerow({
        "Timestamp": timestamp,
        "Epochs": len(history.history['accuracy']),
        "Train_Accuracy": history.history['accuracy'][-1],
        "Val_Accuracy": val_acc,
        "Val_Loss": val_loss,
        "Test_Accuracy": test_acc,
        "Test_Loss": test_loss,
        "Run_Folder_Path": run_folder
    })

print(f"\n✅ Run recorded successfully!")
print(f"📁 Folder created: {run_folder}")
print(f"📊 Training log updated: {csv_file}")
print(f"📦 Model, plots, and results saved with timestamp: {timestamp}")